In [17]:
import pandas as pd
import numpy as np
import csv
import json
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

# load csv data
eval_csv = pd.read_csv("../data/evaluation.csv")
item_csv = pd.read_csv("../data/items.csv", sep="|", quoting=3)
tran_csv = pd.read_csv("../data/transactions.csv", sep="|")
item_with_lang_csv = pd.read_csv("../data/item_with_lang.csv", sep="|", quoting=3)

# replace nan with empty string
item_with_lang_csv = item_with_lang_csv.replace(np.nan, '', regex=True)

# compute frequency of author and publisher
authors_fre = item_with_lang_csv['author'].dropna().value_counts() / len(item_csv)
publishers_fre = item_with_lang_csv['publisher'].dropna().value_counts() / len(item_csv)

# merger eval_csv with item_with_lang_csv
eval_csv = eval_csv.merge(item_with_lang_csv, how='inner', on='itemID')

d = {}
with open("../data/result.txt") as f:
    i = 0 
    for line in f:
        line = line.replace("\'", "\"")
        d2 = json.loads(line)
        item = eval_csv['itemID'].iloc[i]
        d[item] = d2
        i+=1

popularity = tran_csv.drop(['sessionID'], axis=1).groupby(['itemID']).sum().reset_index()
scaler = RobustScaler()
itemID = popularity['itemID']
popularity = popularity.drop('itemID', axis=1)
df_std = pd.DataFrame(scaler.fit_transform(popularity), columns=popularity.columns)
df_std['itemID'] =  itemID

In [87]:
title="Ein Lied, mein Leben und was sonst noch schiefgehen kann"
eval_csv[eval_csv['title']==title]

,itemID,title,author,publisher,main topic,subtopics,lang,lang_prob
6,70766,"Ein Lied, mein Leben und was sonst noch schief...",Tini Wider,tredition,FB,[],de,0.999995


In [88]:
ids = eval_csv[eval_csv['title']==title]['itemID'].values[0]
test_list = list(map(int, d[ids].keys()))[:5]
def compute(click, basket, order):
    return click*0.2 + basket*0.5 + order*0.3
    
diction = {}
list_id = df_std['itemID'].values.tolist()
for item in test_list:
    if item in list_id:
        score = compute(df_std[df_std['itemID']==item]['click'], df_std[df_std['itemID']==item]['basket'], df_std[df_std['itemID']==item]['order']).values[0]
    else:
        score = -10000
    diction[item] = score
    
diction = dict(sorted(diction.items(), reverse=True, key=lambda item: item[1]))
final_list = list(map(int, diction.keys()))
item_with_lang_csv.set_index('itemID').loc[final_list].reset_index()

,itemID,title,author,publisher,main topic,subtopics,lang,lang_prob
0,29136,Vom Regenbogen und dem Fernen Donner,Steffen Unger,tredition,FB,[],de,0.999997
1,68208,WAS SEID IHR nur für Menschen,Charly Michel,tredition,FB,[],de,0.999997
2,15918,WAS SEID IHR nur für Menschen,Charly Michel,tredition,FB,[],de,0.999997
3,33817,Magische Gezeiten - Das Geschenk,Gabi Rüther,tredition,FB,[],de,0.999997
4,34581,Royal Mile - Mein Schritt aus den Schatten,JCR Malnik,tredition,FB,[],de,0.999995
